In [ ]:
import cv2
print(cv2.__version__)

In [1]:
import cv2
#print(cv2.__version__)

img = cv2.imread("C:\img_source\img\himawari.png")
#print(type(img))
#print(img)

cv2.imshow('original', img)
cv2.waitKey()
cv2.destroyAllWindows()

#グレスケに変換
imgGray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

cv2.imshow('Gray Image', imgGray)
cv2.waitKey()
cv2.destroyAllWindows()

################
## ガンマ補正 ##
################

#ガンマ値
gamma = 0.1

# 画素値の最大値
imax = imgGray.max()

# ガンマ補正
imgGamma = imax * (imgGray / imax)**(1/gamma)

cv2.imshow('Gamma Image', imgGamma)
cv2.waitKey()
cv2.destroyAllWindows()